# Generate triplets and questions

In [1]:
from process_data import *

/data1/baixiang/env/anaconda3/envs/edit/lib/python3.9/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


2024-09-07 08:41:18,148 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [2]:
def get_triplets(topic_name):
    with open(f'../data/topic/{topic_name}.json', 'r', encoding='utf-8') as topics_file:
        topics = topics_file.readlines()
    data = []
    # topic_name = 'country'
    # for topic in [f'{{"instance of": "{topic_name}"}}\n']:
    for topic in topics:
        if topic:
            topic = json.loads(topic)
            print(topic)
            query_part1 = "SELECT ?subjectLabel ?relation ?objectLabel WHERE {"
            query_part2 = ""
            relation_object_pairs = convert_topic_to_symbol(topic)
            for pair in relation_object_pairs:
                query_part2 += f"\n?subject wdt:{pair[0]} wd:{pair[1]} ."

            query_part3 = """
                ?subject  ?relation  ?object.
                ?subject wikibase:identifiers ?subject_identifierCount.
                ?object wikibase:identifiers ?object_identifierCount.
                """
            query_part5 = """ 
                FILTER (?subject_identifierCount >= 8 && ?object_identifierCount >= 5) .  
                SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
            }
            LIMIT 50000
            """
            query = query_part1 + query_part2 + query_part3 + query_part5
            sparql.setQuery(query)
            sparql.setReturnFormat(JSON)
            results = sparql.query().convert()
            # print(f"results: {results}")
            if "results" in results:
                # Create a list to store the data
                with ThreadPoolExecutor() as executor:
                    futures = [executor.submit(process_result, result) for result in results["results"]["bindings"]]
                    # Use tqdm to show the progress bar while waiting for all tasks to complete
                    for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
                        result_data = future.result()
                        if result_data:
                            data.append(result_data)

    fact_triplets = pd.DataFrame(data)  # data/triplet/raw/ store raw triplets before removing duplicate (subject, relation) pairs
    fact_triplets.to_csv(f'../data/triplet/raw/{topic_name}.csv', index=False)

In [ ]:
get_triplets('places_city') # 4:37:42
get_triplets('places_landmark') # 4:37:42
get_triplets('entertainment_music_genre') # 28:21

In [ ]:
for topic in ['art_literary', 'art_sculpture', 'health_treatment', 'health_medication', 'health_disease', 'event_film', 'event_sport', 'event_history']:
    get_triplets(topic)

{'instance of': 'literary work'}


HTTPError: HTTP Error 429: Too Many Requests

In [ ]:
topic_name_ls = ['human_scientist', 'human_athlete']
topics_ls = [
  ['{"instance of": "song"}'],
  ['{"instance of": "human", "occupation": "scientist"}\n',],
  ['{"instance of": "human", "occupation": "athlete"}\n',],
  ['{"instance of": "human", "occupation": "entrepreneur"}',],
  # ['{"instance of": "human", "occupation": "actor"}\n'], , 'human_actor'
]

In [9]:
directory = '../data/triplet/raw'
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        fact_triplets = pd.read_csv(f'../data/triplet/raw/{filename}')
        # print(fact_triplets.shape)
        fact_triplets.dropna(inplace=True)
        condition1 = fact_triplets.apply(lambda row: any(val.startswith('Q') and val[1:].isdigit() for val in row.values), axis=1)
        condition2 = fact_triplets.apply(lambda row: any(val.startswith('http') for val in row.values), axis=1)
        fact_triplets = fact_triplets[~(condition1 | condition2)]  

        # remove_pairs, all_pairs = set(), []
        # for i in fact_triplets.index:
        #     subject, relation = fact_triplets.loc[i, 'subjectLabel'], fact_triplets.loc[i, 'relation']
        #     if (subject, relation) in all_pairs:
        #         remove_pairs.add((subject, relation))
        #     else:
        #         all_pairs.append((subject, relation))
        # fact_triplets_new = fact_triplets[~fact_triplets.apply(lambda row: (row['subjectLabel'], row['relation']) in remove_pairs, axis=1)]
        fact_triplets_new = fact_triplets.drop_duplicates(subset=['subjectLabel', 'relation'], keep=False)
        print(f"fact_triplets.shape: {fact_triplets.shape}, fact_triplets_new.shape: {fact_triplets_new.shape}")
        print(f"Check if subject and object are the same: {fact_triplets_new[fact_triplets_new.subjectLabel == fact_triplets_new.objectLabel].shape}")
        fact_triplets_new = fact_triplets_new[fact_triplets_new.subjectLabel != fact_triplets_new.objectLabel]
        fact_triplets_new.to_csv(f'../data/triplet/{filename}', index=False)

fact_triplets.shape: (44957, 3), fact_triplets_new.shape: (18995, 3)
Check if subject and object are the same: (482, 3)
fact_triplets.shape: (3007, 3), fact_triplets_new.shape: (1652, 3)
Check if subject and object are the same: (19, 3)
fact_triplets.shape: (5227, 3), fact_triplets_new.shape: (2654, 3)
Check if subject and object are the same: (25, 3)
fact_triplets.shape: (4999, 3), fact_triplets_new.shape: (2402, 3)
Check if subject and object are the same: (10, 3)
fact_triplets.shape: (627, 3), fact_triplets_new.shape: (360, 3)
Check if subject and object are the same: (1, 3)
fact_triplets.shape: (7531, 3), fact_triplets_new.shape: (3202, 3)
Check if subject and object are the same: (18, 3)
fact_triplets.shape: (4999, 3), fact_triplets_new.shape: (1794, 3)
Check if subject and object are the same: (6, 3)
fact_triplets.shape: (190, 3), fact_triplets_new.shape: (92, 3)
Check if subject and object are the same: (0, 3)
fact_triplets.shape: (9997, 3), fact_triplets_new.shape: (4339, 3)
Ch

In [23]:
remove_relation = ["topic's main category", "topic's main template", "described by source", "Commons category", "on focus list of Wikimedia project"]

directory = '../data/triplet'
for filename in os.listdir(directory)[:]:
    if filename.endswith('.csv'):
        fact_triplets_new = pd.read_csv(f'../data/triplet/{filename}')
        
        with open(os.path.join("../data/topic", filename.replace('.csv', '.json')), 'r', encoding='utf-8') as topics_file:
            topics = topics_file.readlines()
        topic = json.loads(topics[0])
        first_pair = next(iter(topic.items()))
        print(f'filename: {filename}, topic: {first_pair[1]}')

        question_ls = []
        for i in tqdm(fact_triplets_new.index[:]):
            subject, relation, object = fact_triplets_new.loc[i, 'subjectLabel'], fact_triplets_new.loc[i, 'relation'], fact_triplets_new.loc[i, 'objectLabel']
            question = generate_question(subject, relation, object, first_pair[1])
            if question:
                question_ls.append(question)
        df = pd.DataFrame(question_ls)
        df.to_csv(f"../data/questions/unfiltered/{filename}", index=False) # [['subject', 'relation', 'object', 'question']]
        print(f"fact_triplets_new.shape: {fact_triplets_new.shape} df.shape: {df.shape}")
        print(f"Check if object is not equal to label: {df[df.object != df.label].shape}")

filename: places_city.csv, topic: city


  0%|          | 0/18513 [00:00<?, ?it/s]

100%|██████████| 18513/18513 [03:08<00:00, 98.37it/s] 


fact_triplets_new.shape: (18513, 3) df.shape: (6805, 5)
Check if object is not equal to label: (0, 5)
filename: entertainment_music_genre.csv, topic: music genre


100%|██████████| 1633/1633 [00:17<00:00, 92.27it/s]


...

fact_triplets_new.shape: (4339, 3) df.shape: (2687, 5)
Check if object is not equal to label: (0, 5)
filename: places_country.csv, topic: country


100%|██████████| 2761/2761 [00:30<00:00, 90.19it/s]


fact_triplets_new.shape: (2761, 3) df.shape: (1936, 5)
Check if object is not equal to label: (0, 5)
filename: human_entrepreneur.csv, topic: human


100%|██████████| 2592/2592 [00:26<00:00, 96.65it/s] 

fact_triplets_new.shape: (2592, 3) df.shape: (1546, 5)
Check if object is not equal to label: (0, 5)
